# Data Collection

### Downloading data

In [1]:
# python download_data.py

### Combining data from all chunks into one file

In [2]:
# cat data/offers_data*.txt | grep -v '"error"' > data/offers_data.txt

# Data Cleaning & Preparation

### Importing libraries and configuring settings

In [3]:
import numpy as np
import pandas as pd

In [4]:
pd.set_option('display.max_columns', None)

### Loading data and reviewing data

In [5]:
df = pd.read_json('data/offers_data.txt', lines=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91712 entries, 0 to 91711
Data columns (total 36 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   street              54322 non-null  object 
 1   equipment           27034 non-null  object 
 2   areas               79487 non-null  object 
 3   floor               89648 non-null  object 
 4   kitchen             17296 non-null  object 
 5   parking             0 non-null      float64
 6   numberOfRooms       91712 non-null  int64  
 7   rooms               0 non-null      float64
 8   type                72648 non-null  object 
 9   windowsOrientation  0 non-null      float64
 10  year                67677 non-null  float64
 11  material            44438 non-null  object 
 12  windows             54138 non-null  object 
 13  heating             65621 non-null  object 
 14  numberOfFloors      83861 non-null  float64
 15  conveniences        58185 non-null  object 
 16  secu

In [7]:
df.isnull().mean().sort_values(ascending=False)

parking               1.000000
rooms                 1.000000
windowsOrientation    1.000000
free_from             0.854915
kitchen               0.811410
equipment             0.705229
rent_currency         0.603138
rent                  0.603138
media_types           0.573349
material              0.515461
security              0.443999
windows               0.409696
street                0.407689
conveniences          0.365568
heating               0.284488
year                  0.262070
ownership             0.241604
condition             0.233001
type                  0.207868
areas                 0.133298
numberOfFloors        0.085605
lift                  0.079870
ad_price              0.053232
floor                 0.022505
long                  0.000000
price_currency        0.000000
city_name             0.000000
lat                   0.000000
region_name           0.000000
subregion_id          0.000000
area                  0.000000
propertype            0.000000
user_typ

- renaming columns

In [8]:
df['number_of_rooms'] = df.numberOfRooms
df['number_of_floors'] = df.numberOfFloors
df['building_type'] = df.type
df['windows_orientation'] = df.windowsOrientation
df = df.drop(columns=['numberOfRooms', 'numberOfFloors', 'type', 'windowsOrientation'])

### Imputing missing values

#### equipment, areas, conveniences, security, media_types

**Analysis:**
- columns with list-like comma-separated string values

**Decision:**
- create separate column for each unique characteristic
- treat null values as lacking of specific characteristic

In [9]:
multiple_values_columns = ['equipment', 'areas', 'conveniences', 'security', 'media_types']
for column in multiple_values_columns:
    types = df[column].dropna().apply(lambda x: x.split(', ') if x else None).explode().unique()
    for t in types:
        df[t] = df[column].apply(lambda x: 1 if x is not None and t in x else 0)
df = df.drop(columns=multiple_values_columns)

- removing duplicated columns added during creation

In [10]:
(df['INTERNET'] == df['internet']).all()

True

In [11]:
df = df.drop(columns=['internet'])

In [12]:
df.loc[df['lift'].isnull(), 'lift'] = df['LIFT']
df['lift'].replace({'y': 1, 'n': 0}, inplace=True)
(df['LIFT'] == df['lift']).all()

True

In [13]:
df = df.drop(columns=['lift'])

#### kitchen

In [14]:
df.kitchen.unique()

array([None, 'SEPARATE'], dtype=object)

**Analysis:**
- only one unique value is present: SEPERATE

**Decision:**
- create separate_kitchen column with values:
  - 0 - non-separate
  - 1 - separate 
- treat null values as 0 - non-separate

In [15]:
df['separate_kitchen'] = df.kitchen.map(lambda k: 1 if k else 0)
df = df.drop(columns=['kitchen'])

### heating

In [16]:
building_type_grp = df.groupby(['building_type'])
building_type_grp['heating'].value_counts(normalize=True)

building_type  heating    
APARTMENT      URBAN          0.735046
               GAS            0.148202
               OTHER          0.071283
               BOILER_ROOM    0.028369
               ELECTRICAL     0.016969
               TILED_STOVE    0.000131
BLOCK          URBAN          0.806867
               GAS            0.114169
               OTHER          0.046790
               BOILER_ROOM    0.022180
               ELECTRICAL     0.008734
               TILED_STOVE    0.001261
HOUSE          GAS            0.540778
               OTHER          0.210790
               URBAN          0.114806
               BOILER_ROOM    0.064617
               ELECTRICAL     0.056462
               TILED_STOVE    0.012547
INFILL         URBAN          0.855263
               GAS            0.078947
               BOILER_ROOM    0.039474
               OTHER          0.013158
               ELECTRICAL     0.013158
LOFT           URBAN          0.808219
               GAS            0.16438

**Analysis:**
- 28% of values are null
- different building types generally exhibit a preference for specific heating types over others

**Decision**: 
- fillna using the mode value associated with each building type
- if obtaining the mode value for a specific building type is not possible, use the mode value corresponding to the entire dataset

In [17]:
for building_type in df.building_type.dropna().unique():
    heating_mode = df[df['building_type'] == building_type]['heating'].mode()
    heating_mode = heating_mode[0] if heating_mode.any() else df['heating'].mode()[0]
    df.loc[df['building_type'] == building_type, 'heating'] = df['heating'].fillna(heating_mode)

### windows, ownership

In [18]:
df.windows.value_counts(normalize=True)

windows
PLASTIC      0.920444
WOODEN       0.065259
ALUMINIUM    0.014297
Name: proportion, dtype: float64

In [19]:
df.ownership.value_counts(normalize=True)

ownership
FULL_OWNERSHIP            0.948874
LIMITED_OWNERSHIP         0.046266
SHARE                     0.004026
USUFRUCT                  0.000820
CO_OPERATIVE_OWNERSHIP    0.000014
Name: proportion, dtype: float64

**Analysis:**
- one value significantly dominates, representing over 90% of occurrences compared to the others

**Decision:** 
- fillna using the mode value corresponding to the entire dataset

In [20]:
df.windows.fillna(df.windows.mode()[0], inplace=True)
df.ownership.fillna(df.ownership.mode()[0], inplace=True)

### parking, rooms, windows_orientation, free_from, rent, rent_currency, material, street, year, condition, building_type, number_of_floors, ad_price, floor

**Analysis:**
- no resonable methods of imputation where identified

## Cleaning data

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91712 entries, 0 to 91711
Data columns (total 59 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   street               54322 non-null  object 
 1   floor                89648 non-null  object 
 2   parking              0 non-null      float64
 3   rooms                0 non-null      float64
 4   year                 67677 non-null  float64
 5   material             44438 non-null  object 
 6   windows              91712 non-null  object 
 7   heating              77767 non-null  object 
 8   condition            70343 non-null  object 
 9   ownership            91712 non-null  object 
 10  rent                 36397 non-null  float64
 11  rent_currency        36397 non-null  object 
 12  lat                  91712 non-null  float64
 13  long                 91712 non-null  float64
 14  ad_price             86830 non-null  float64
 15  price_currency       91712 non-null 

In [22]:
df.isnull().mean().sort_values(ascending=False)

parking                1.000000
rooms                  1.000000
windows_orientation    1.000000
free_from              0.854915
rent                   0.603138
rent_currency          0.603138
material               0.515461
street                 0.407689
year                   0.262070
condition              0.233001
building_type          0.207868
heating                0.152052
number_of_floors       0.085605
ad_price               0.053232
floor                  0.022505
garage                 0.000000
garden                 0.000000
ENTRYPHONE             0.000000
terrace                0.000000
usable_room            0.000000
INTERNET               0.000000
LIFT                   0.000000
MONITORING             0.000000
ALARM                  0.000000
CLOSED_AREA            0.000000
ROLLER_SHUTTERS        0.000000
ANTI_BURGLARY_DOOR     0.000000
balcony                0.000000
cable-television       0.000000
phone                  0.000000
cable_television       0.000000
electric

##### Analysis:
- parking, rooms, windows_orientation, free_from, rent, rent_currency, material - over 50% of null values
- propertype, offertype - only one unique value
- street - 41% null values
- year, condition, building_type, heating - less than 30% of null values
- number_of_floors, ad_price, floor - less than 1% of null values
##### Decision:
- parking, rooms, windows_orientation, free_from, rent, rent_currency, material, propertype, offertype - drop entire columns
- street, year, condition, building_type, heating, number_of_floors, ad_price, floor - dropna

In [23]:
df = df.drop(columns=['parking', 'rooms', 'windows_orientation', 'free_from', 'rent', 'rent_currency', 'material', 'propertype', 'offertype'])

In [24]:
df.dropna(inplace=True)
df.reset_index(inplace=True)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26230 entries, 0 to 26229
Data columns (total 51 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               26230 non-null  int64  
 1   street              26230 non-null  object 
 2   floor               26230 non-null  object 
 3   year                26230 non-null  float64
 4   windows             26230 non-null  object 
 5   heating             26230 non-null  object 
 6   condition           26230 non-null  object 
 7   ownership           26230 non-null  object 
 8   lat                 26230 non-null  float64
 9   long                26230 non-null  float64
 10  ad_price            26230 non-null  float64
 11  price_currency      26230 non-null  object 
 12  city_name           26230 non-null  object 
 13  market              26230 non-null  object 
 14  region_name         26230 non-null  object 
 15  subregion_id        26230 non-null  object 
 16  area